In [1]:
# import the necessary packages
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from sklearn.preprocessing import LabelBinarizer
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from keras.optimizers import Adam
import pandas as pd
import numpy as np
import glob
import cv2
import os
import locale

In [2]:
cols = ["bedrooms", "bathrooms", "area", "zipcode", "price"]
df = pd.read_csv("https://raw.githubusercontent.com/emanhamed/Houses-dataset/master/Houses%20Dataset/HousesInfo.txt", sep=" ", header=None, names=cols)


In [3]:
df.head()

,bedrooms,bathrooms,area,zipcode,price
0,4,4.0,4053,85255,869500
1,4,3.0,3343,36372,865200
2,3,4.0,3923,85266,889000
3,5,5.0,4022,85262,910000
4,3,4.0,4116,85266,971226


In [4]:
zipcodes, counts = np.unique(df["zipcode"], return_counts=True)

In [5]:
df.shape

(535, 5)

In [6]:
# loop over each of the unique zip codes and their corresponding
# count
for (zipcode, count) in zip(zipcodes, counts):
    # the zip code counts for our housing dataset is *extremely*
    # unbalanced (some only having 1 or 2 houses per zip code)
    # so let's sanitize our data by removing any houses with less
    # than 25 houses per zip code
    if count < 25:
        idxs = df[df["zipcode"] == zipcode].index
        df.drop(idxs, inplace=True)

In [7]:
df.shape

(362, 5)

In [8]:
(train, test) = train_test_split(df, test_size=0.25, random_state=42)
print(train.shape)
print(test.shape)

(271, 5)
(91, 5)


In [9]:
# find the largest house price in the training set and use it to
# scale our house prices to the range [0, 1] (this will lead to
# better training and convergence)
maxPrice = train["price"].max()
trainY = train["price"] / maxPrice
testY = test["price"] / maxPrice

In [10]:
# initialize the column names of the continuous data
continuous = ["bedrooms", "bathrooms", "area"]

# performin min-max scaling each continuous feature column to
# the range [0, 1]
cs = MinMaxScaler()
trainContinuous = cs.fit_transform(train[continuous])
testContinuous = cs.transform(test[continuous])

In [11]:
# one-hot encode the zip code categorical data (by definition of
# one-hot encoing, all output features are now in the range [0, 1])
zipBinarizer = LabelBinarizer().fit(df["zipcode"])
trainCategorical = zipBinarizer.transform(train["zipcode"])
testCategorical = zipBinarizer.transform(test["zipcode"])

In [12]:
# construct our training and testing data points by concatenating
# the categorical features with the continuous features
trainX = np.hstack([trainCategorical, trainContinuous])
testX = np.hstack([testCategorical, testContinuous])

print(trainX.shape)
print(testX.shape)

(271, 10)
(91, 10)


In [13]:
dim = trainX.shape[1]
# define our MLP network
model = Sequential()
model.add(Dense(8, input_dim=dim, activation="relu"))
model.add(Dense(4, activation="relu"))
model.add(Dense(1, activation="linear"))

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [14]:
opt = Adam(learning_rate=1e-3)
model.compile(loss="mean_absolute_percentage_error", optimizer=opt)

In [15]:
model.fit(trainX, trainY, validation_data=(testX, testY),
    epochs=200, batch_size=8)

Epoch 1/200
34/34 ━━━━━━━━━━━━━━━━━━━━ 7s 62ms/step - loss: 339.5714 - val_loss: 132.6852
Epoch 2/200
34/34 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - loss: 104.7882 - val_loss: 39.2724
Epoch 3/200
34/34 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 47.9132 - val_loss: 34.3352
Epoch 4/200
34/34 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 43.1915 - val_loss: 33.6869
Epoch 5/200
34/34 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 41.8978 - val_loss: 31.8707
Epoch 6/200
34/34 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 37.3424 - val_loss: 32.2165
Epoch 7/200
34/34 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 38.8275 - val_loss: 31.0144
Epoch 8/200
34/34 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 37.5898 - val_loss: 31.2372
Epoch 9/200
34/34 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 34.0325 - val_loss: 29.1514
Epoch 10/200
34/34 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 34.8190 - val_loss: 32.8828
Epoch 11/200
34/34 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 36.7833 - val_loss: 30.4664
Epoch 12/200
34/34 ━━━━━━━━━━━━━━━

In [16]:
preds = model.predict(testX)

3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 127ms/step


In [17]:
# make predictions on the testing data
preds = model.predict(testX)

# compute the difference between the *predicted* house prices and the
# *actual* house prices, then compute the percentage difference and
# the absolute percentage difference
diff = preds.flatten() - testY
percentDiff = (diff / testY) * 100
absPercentDiff = np.abs(percentDiff)

# compute the mean and standard deviation of the absolute percentage
# difference
mean = np.mean(absPercentDiff)
std = np.std(absPercentDiff)

# finally, show some statistics on our model
locale.setlocale(locale.LC_ALL, "en_US.UTF-8")
print("avg. house price: {}, std house price: {}".format(
    locale.currency(df["price"].mean(), grouping=True),
    locale.currency(df["price"].std(), grouping=True)))
print("mean: {:.2f}%, std: {:.2f}%".format(mean, std))

3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step 
avg. house price: $533,388.27, std house price: $493,403.08
mean: 23.67%, std: 20.59%
